<a href="https://colab.research.google.com/github/lamesnake/EYE-CANE/blob/main/learning_bert_for_the_first_time.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## This is the very first time I would be implementing BERT.
The Kernels which I fould very helpful for implementing BERT are given below. These kernels really helped to understand and implement the model. You might find my kernel almost similar to them, and I openly accent this fact that what ever is there in this kernel is 99% of their contribution.
* [Disaster NLP: Keras BERT using TFHub](https://www.kaggle.com/xhlulu/disaster-nlp-keras-bert-using-tfhub) by [xhlulu](https://www.kaggle.com/xhlulu)
* [BERT for Humans: Tutorial+Baseline](https://www.kaggle.com/abhinand05/bert-for-humans-tutorial-baseline) by [Abhinand](https://www.kaggle.com/abhinand05)

On the Internet and kaggle there are various pre-trained model of bert in keras and pytorch. But, I choose these kernels because I found them easy to understand and implement. For understanding purpose,I basically prefered the kernel **BERT for Humans: Tutorial+Baseline**, but for implementation purpose I found the kernel **Disaster NLP: Keras BERT using TFHub** to be very useful. I would be explaining in sort but for detailed understanding plese visit their beautiful kernels.

## What is bert? (According to me)
In 2019 a model ruled over the entire NLP dimension and that is bert. Bert is not the solution to very NLP problem, but yes it does gives you a upper hand when it comes down to accuracy.<br>
To understand bert you need to understand certain basic terms sunch as bidirectional encoder, transformer, look-ahead masking, transformer, and attention making(both multihead and self) in the world of NLP.
For better understanding I would recommend you this article at Analytics Vidya(Link - https://www.analyticsvidhya.com/blog/2019/06/understanding-transformers-nlp-state-of-the-art-models/)

In [10]:
# We will use the official tokenization script created by the Google team
!wget --quiet https://raw.githubusercontent.com/tensorflow/models/master/official/nlp/bert/tokenization.py

In [11]:
import numpy as np
import pandas as pd
import re
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint
import tensorflow_hub as hub
#from nltk.corpus import stopwords
#from nltk.stem.porter import PorterStemmer

from bert import tokenization

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [12]:
BASE_PATH = "/kaggle/input/nlp-getting-started/"

In [24]:
train =pd.read_csv("/content/sample_data/california_housing_train.csv")
train.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
0,-114.31,34.19,15.0,5612.0,1283.0,1015.0,472.0,1.4936,66900.0
1,-114.47,34.40,19.0,7650.0,1901.0,1129.0,463.0,1.8200,80100.0
2,-114.56,33.69,17.0,720.0,174.0,333.0,117.0,1.6509,85700.0
3,-114.57,33.64,14.0,1501.0,337.0,515.0,226.0,3.1917,73400.0
4,-114.57,33.57,20.0,1454.0,326.0,624.0,262.0,1.9250,65500.0


In [25]:
test =pd.read_csv("/content/sample_data/california_housing_test.csv")
test.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
0,-122.05,37.37,27.0,3885.0,661.0,1537.0,606.0,6.6085,344700.0
1,-118.30,34.26,43.0,1510.0,310.0,809.0,277.0,3.5990,176500.0
2,-117.81,33.78,27.0,3589.0,507.0,1484.0,495.0,5.7934,270500.0
3,-118.36,33.82,28.0,67.0,15.0,49.0,11.0,6.1359,330000.0
4,-119.67,36.33,19.0,1241.0,244.0,850.0,237.0,2.9375,81700.0


In [26]:
%%time
module_url = "https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1"
bert_layer = hub.KerasLayer(module_url, trainable=True)

CPU times: user 11.3 s, sys: 3.53 s, total: 14.8 s
Wall time: 16.6 s


In [27]:
vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
do_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = tokenization.FullTokenizer(vocab_file, do_lower_case)

## Notes
So bert actually takes three inputs, first is id of tokenized text, second is padding ID and third is Segmentation ID. We have added the tokens "[CLS]" and "[SEP]" at the beginning and end of a sentence maker it easier for the model to understand the beginning and end of a senntence.<br>
In the cell below the entire pre-processing of text before feeding to the bert is explained with code.

In [28]:
text = "This is a Goat, and I am riding a Boat...."
tokenize_ = tokenizer.tokenize(text)
print("Text after tokenization: ")
print(tokenize_)
max_len = 25

text = tokenize_[:max_len-2]
input_sequence = ["[CLS]"] + text + ["[SEP]"]
pad_len = max_len - len(input_sequence)

print("After adding [CLS] and [SEP]: ")
print(input_sequence)
tokens = tokenizer.convert_tokens_to_ids(input_sequence)
print("After converting Tokens to Id: ")
print(tokens)
tokens += [0] * pad_len
print("tokens: ")
print(tokens)
pad_masks = [1] * len(input_sequence) + [0] * pad_len
print("Pad Masking: ")
print(pad_masks)
segment_ids = [0] * max_len
print("Segment Ids: ")
print(segment_ids)

UnparsedFlagAccessError: ignored

In [ ]:
def pre_Process_data(documents, tokenizer, max_len=512):
    '''
    For preprocessing we have regularized, transformed each upper case into lower case, tokenized,
    Normalized and remove stopwords. For normalization, we have used PorterStemmer. Porter stemmer transforms 
    a sentence from this "love loving loved" to this "love love love"
    
    '''
    all_tokens = []
    all_masks = []
    all_segments = []
    print("Pre-Processing the Data.........\n")
    for data in documents:
        review = re.sub('[^a-zA-Z]', ' ', data)
        url = re.compile(r'https?://\S+|www\.\S+')
        review = url.sub(r'',review)
        html=re.compile(r'<.*?>')
        review = html.sub(r'',review)
        emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
        review = emoji_pattern.sub(r'',review)
        text = tokenizer.tokenize(review)
        text = text[:max_len-2]
        input_sequence = ["[CLS]"] + text + ["[SEP]"]
        pad_len = max_len - len(input_sequence)
        
        tokens = tokenizer.convert_tokens_to_ids(input_sequence)
        tokens += [0] * pad_len
        pad_masks = [1] * len(input_sequence) + [0] * pad_len
        segment_ids = [0] * max_len
        
        all_tokens.append(tokens)
        all_masks.append(pad_masks)
        all_segments.append(segment_ids)
    return np.array(all_tokens), np.array(all_masks), np.array(all_segments)

In [ ]:
input_word_id = Input(shape=(max_len,),dtype=tf.int32, name="input_word_ids")
input_mask = Input(shape=(max_len,), dtype=tf.int32, name="input_mask")
segment_id = Input(shape=(max_len,), dtype=tf.int32, name = "segment_id")

_, sequence_output = bert_layer([input_word_id, input_mask, segment_id])
clf_output = sequence_output[:, 0, :]
model = Model(inputs=[input_word_id, input_mask, segment_id],outputs=clf_output)
model.compile(Adam(lr=2e-5), loss='binary_crossentropy', metrics=['accuracy'])
model.summary()
print("shape of _ layer of BERT: "+str(_.shape))
print("shape of last layer of BERT: "+str(sequence_output.shape))

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_word_ids (InputLayer)     [(None, 25)]         0                                            
__________________________________________________________________________________________________
input_mask (InputLayer)         [(None, 25)]         0                                            
__________________________________________________________________________________________________
segment_id (InputLayer)         [(None, 25)]         0                                            
__________________________________________________________________________________________________
keras_layer (KerasLayer)        [(None, 768), (None, 109482241   input_word_ids[0][0]             
                                                                 input_mask[0][0]             

In [ ]:
def build_model(bert_layer, max_len=512):
    input_word_id = Input(shape=(max_len,),dtype=tf.int32, name="input_word_ids")
    input_mask = Input(shape=(max_len,), dtype=tf.int32, name="input_mask")
    segment_id = Input(shape=(max_len,), dtype=tf.int32, name = "segment_id")
    
    _, sequence_output = bert_layer([input_word_id, input_mask, segment_id])
    clf_output = sequence_output[:, 0, :]
    dense_layer1 = Dense(units=256,activation='relu')(clf_output)
    dense_layer1 = Dropout(0.4)(dense_layer1)
    dense_layer2 = Dense(units=128, activation='relu')(dense_layer1)
    dense_layer2 = Dropout(0.4)(dense_layer2)
    out = Dense(1, activation='sigmoid')(dense_layer2)
    
    model = Model(inputs=[input_word_id, input_mask, segment_id],outputs=out)
    model.compile(Adam(lr=2e-5), loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

In [ ]:
train_input = pre_Process_data(train.text.values, tokenizer, max_len=260)
test_input = pre_Process_data(test.text.values, tokenizer, max_len=260)
train_labels = train.target.values

Pre-Processing the Data.........

Pre-Processing the Data.........



In [ ]:
model = build_model(bert_layer, max_len=260)
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_word_ids (InputLayer)     [(None, 260)]        0                                            
__________________________________________________________________________________________________
input_mask (InputLayer)         [(None, 260)]        0                                            
__________________________________________________________________________________________________
segment_id (InputLayer)         [(None, 260)]        0                                            
__________________________________________________________________________________________________
keras_layer (KerasLayer)        [(None, 768), (None, 109482241   input_word_ids[0][0]             
                                                                 input_mask[0][0]           

In [ ]:
checkpoint = ModelCheckpoint('model.h5', monitor='val_loss', save_best_only=True)
train_history = model.fit(
    train_input, train_labels,
    validation_split=0.2,
    epochs=10,
    callbacks=[checkpoint],
    batch_size=32
)

Train on 6090 samples, validate on 1523 samples
Epoch 1/10
6090/6090 [==============================] - 211s 35ms/sample - loss: 0.4809 - accuracy: 0.7805 - val_loss: 0.3833 - val_accuracy: 0.8365
Epoch 2/10
6090/6090 [==============================] - 186s 31ms/sample - loss: 0.3372 - accuracy: 0.8745 - val_loss: 0.3997 - val_accuracy: 0.8306
Epoch 3/10
6090/6090 [==============================] - 186s 30ms/sample - loss: 0.2311 - accuracy: 0.9186 - val_loss: 0.4222 - val_accuracy: 0.8299
Epoch 4/10
6090/6090 [==============================] - 186s 30ms/sample - loss: 0.1323 - accuracy: 0.9504 - val_loss: 0.7445 - val_accuracy: 0.7984
Epoch 5/10
6090/6090 [==============================] - 186s 30ms/sample - loss: 0.0866 - accuracy: 0.9703 - val_loss: 0.7747 - val_accuracy: 0.8207
Epoch 6/10
6090/6090 [==============================] - 186s 30ms/sample - loss: 0.0701 - accuracy: 0.9739 - val_loss: 0.9173 - val_accuracy: 0.8063
Epoch 7/10
6090/6090 [==============================] - 18

In [ ]:
submission = pd.read_csv(BASE_PATH + "sample_submission.csv")
submission.head()

,id,target
0,0,0
1,2,0
2,3,0
3,9,0
4,11,0


In [ ]:
model.load_weights('model.h5')
test_pred = model.predict(test_input)

submission['target'] = test_pred.round().astype(int)
submission.to_csv('submission.csv', index=False)



## So, is this cheating....
As this is a competition, and I have used idea from someones kernel who are also in the same competition. So in one way, I won't deny the fact.....<br>
But I have struggled to find ways to implement bert. So many pre-trained models on Github, so many models and kernels on kaggle, but no starting point. I feel that many of you kagglers and non-kagglers have passed through this phase or maybe passing through this phase, to them this kernel would be very much helpful. This kernel would act as a great starting point for them.

## Thank You